**<h1><center>PROJECT DATA ANALYSIS</center></h1>**

Thành viên:

|STT|MSSV|Họ tên|
|---|---|---|
|1|||
|2|||
|3|||

# 0. Chọn nguồn dữ liệu

Link: https://catalog.data.gov/dataset/local-weather-archive

# 1. Thuyết minh dữ liệu

Bộ dữ liệu này chứa dữ liệu thời tiết sân bay quốc tế `Raleigh Durham` được lấy từ dịch vụ web `NOAA` được mô tả tại dữ liệu khí hậu trực tuyến: Tài liệu dịch vụ web. 

Chúng tôi đã rút dữ liệu này và chuyển đổi nó thành các đơn vị thường được sử dụng. 

Bộ dữ liệu này là một kho lưu trữ - nó không được cập nhật.

Thông tin thuộc tính

|Thuộc tính|Mô tả|Đơn vị|
|---|---|---|
|date|Thời gian|năm-tháng-ngày|
|tmin|Nhiệt độ thấp nhất|$^oC$|
|tmax|Nhiệt độ cao nhất|$^oC$|
|prcp|Lượng mưa|mm|
|snow|Lượng tuyết rơi|cm|
|snwd|Độ sâu tuyết|cm|
|awnd|Tốc độ gió trung bình|kph|

# 2. Đặt câu hỏi tình huống, vấn đề tìm hiểu trên bộ dữ liệu

1. Nhiệt độ cao nhất/thấp nhất của mỗi năm là bao nhiêu?
2. Mưa nhiều thường tập trung khoảng thời gian (tháng) nào trong năm?

# 3. Thực hiện các vấn đề

In [21]:
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

Đọc dữ liệu

In [2]:
df = pd.read_csv('weather-history.csv', parse_dates =["date"], index_col ="date") # , index_col ="date"
df.head()

,tmin,tmax,prcp,snow,snwd,awnd
date,,,,,,
2017-01-09,9,31,0.00,0.0,1.2,2.46
2017-01-11,40,57,0.00,0.0,0.0,6.04
2017-01-19,36,63,0.00,0.0,0.0,1.34
2017-01-20,46,59,0.09,0.0,0.0,2.91
2017-01-24,41,63,0.00,0.0,0.0,6.04


In [32]:
y2017 = df[df.index.year == 2021]
yy = y2017.resample('M').sum()
yy.index.month

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int64', name='date')

In [4]:
ele = {'tmin': 'Nhiệt độ thấp nhất', 'tmax': 'Nhiệt độ cao nhất', 'prcp': 'Lượng mưa', 'snwd': 'Lượng tuyết rơi', 'awnd': 'Tốc độ gió trung bình'}

Xem số lượng dòng, cột của tập dữ liệu

In [5]:
df.shape

(1977, 6)

Xem kiểu dữ liệu các trường dữ liệu

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1977 entries, 2017-01-09 to 2022-05-22
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tmin    1977 non-null   int64  
 1   tmax    1977 non-null   int64  
 2   prcp    1977 non-null   float64
 3   snow    1977 non-null   float64
 4   snwd    1976 non-null   float64
 5   awnd    1973 non-null   float64
dtypes: float64(4), int64(2)
memory usage: 108.1 KB


Kiểm tra dữ liệu `NaN`

In [7]:
df.isna().sum()

tmin    0
tmax    0
prcp    0
snow    0
snwd    1
awnd    4
dtype: int64

Điền dữ liệu `NaN` bằng những giá trị phía dưới (tức ngày hôm sau)

In [8]:
df.fillna(method='bfill', inplace=True)
df.isna().sum()

tmin    0
tmax    0
prcp    0
snow    0
snwd    0
awnd    0
dtype: int64

Xem thống kê tổng quan

In [9]:
df.describe()

,tmin,tmax,prcp,snow,snwd,awnd
count,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000
mean,51.088012,72.298938,0.135771,0.012838,0.016945,6.075488
std,15.889527,15.796087,0.370521,0.227433,0.215284,2.952269
min,4.000000,27.000000,0.000000,0.000000,0.000000,0.000000
25%,37.000000,60.000000,0.000000,0.000000,0.000000,3.800000
50%,52.000000,74.000000,0.000000,0.000000,0.000000,5.800000
75%,66.000000,86.000000,0.060000,0.000000,0.000000,7.800000
max,78.000000,102.000000,4.960000,7.000000,5.900000,20.360000


trực quan

### tổng quan

In [ ]:
val = 'sum'
yearly = None
match val:
    case 'sum':
        yearly = df.resample('Y').sum()
    case 'min': 
        yearly = df.resample('Y').min()
    case 'max':
        yearly = df.resample('Y').max()
    case 'mean': 
        yearly = df.resample('Y').mean()
    case 'median': 
        yearly = df.resample('Y').median()

Nhiệt độ

In [57]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=yearly.index.year, y=yearly['tmin'],
    name=ele['tmin'],
    connectgaps=True
))

fig.add_trace(go.Scatter(x=yearly.index.year, y=yearly['tmax'],
    name=ele['tmax'],
    connectgaps=True,
))
fig.update_layout(title_text=f'Tổng nhiệt độ của từng năm')
fig.show()

Lượng mưa

In [75]:
fig = go.Figure()
fig.add_trace(go.Bar(x = yearly.index.year, y=yearly.prcp, text = yearly.prcp, textposition='auto'))
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text=f'Tổng lượng mưa của từng năm')
fig.show()

Lượng tuyết rơi

In [80]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=yearly.index.year, values=yearly.snow, name="Lượng tuyết rơi"), 1, 1)
fig.add_trace(go.Pie(labels=yearly.index.year, values=yearly.snwd, name="Độ dày tuyết"), 1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Tổng lượng tuyết rơi và độ dày tuyết hàng năm",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='SNOW', x=0.2, y=0.5, font_size=20, showarrow=False),
                 dict(text='DEPTH', x=0.8, y=0.5, font_size=20, showarrow=False)])
fig.show()

Tốc độ gió trung bình

In [81]:
fig = px.line_polar(yearly, r=yearly.awnd, theta=yearly.index.year, line_close=True)
fig.update_traces(fill='toself')
fig.update_layout(title = 'Tốc độ gió trung bình hàng năm')
fig.show()

C:\Users\quach\AppData\Roaming\Python\Python310\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [54]:
year = df[df.index.year == 2020]
fig = go.Figure()
fig.add_trace(go.Histogram(x = df['tmin'], name = ele['tmin']))
fig.add_trace(go.Histogram(x = df['tmax'], name = ele['tmax']))
# Overlay both histograms
fig.update_layout(barmode='overlay', title_text=f'Nhiệt độ trong năm {year.index.year[0]}')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
# fig.update_layout(
#     title_text=f'Nhiệt độ trong năm {year.index.year[0]}', # title of plot
#     xaxis_title_text='Month', # xaxis label
#     yaxis_title_text='Count', # yaxis label
#     bargap=0.2, # gap between bars of adjacent location coordinates
#     bargroupgap=0.1 # gap between bars of the same location coordinates
# )
fig.show()

In [36]:
fig = px.pie(yearly, values='snow', title='Tổng lượng tuyết rơi từng năm', hole=.3, names=yearly.index.year)
fig.show()

In [33]:
year = 2021
df_y = df[df.index.year == year]
monthly = df_y.resample('M').sum()
fig = px.bar(monthly.index.month, y=monthly['prcp'], title = f'Biểu đồ tổng lượng mưa năm {year}', text_auto=True)
fig.show()

# 4. Thực hiện thông qua một hoặc hai dashboard

In [12]:
df.columns

Index(['tmin', 'tmax', 'prcp', 'snow', 'snwd', 'awnd'], dtype='object')

In [29]:
from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
# app = Dash(__name__)


app.layout = html.Div([
    html.H1(children='Dash Board', style = {'text-align': 'center'}),

    dbc.Row([
        html.H6('Chọn thuộc tính'),
        dbc.Col([
                
                dcc.Dropdown(id="element", 
                         options=[{"label": "Nhiệt độ thấp nhất", "value": "tmin"},
                                  {"label": "Nhiệt độ thấp nhất", "value": "tmax"},
                                  {"label": "Lượng mưa", "value": "prcp"},
                                  {"label": "Lượng tuyết rơi", "value": "snow"},
                                  {"label": "Độ sâu lớp tuyết", "value": "snwd"},
                                  {"label": "Tốc độ gió trung bình", "value": "awnd"}],
                         multi=False,
                         value="prcp",
                         style={'width': "40%"})
        ])
    ]),

    dbc.Row([
        dcc.Graph(id="graph1")
    ])
])

@app.callback(Output("graph1", "figure"), 
              Input("element", "value"))
def display_graph(element):
    yearly = df.resample('Y').sum()
    # fig = px.bar(yearly, y=element, title = f'Biểu đồ {element} theo từng năm')
    fig = go.Figure(
        data=[go.Bar(x=yearly.index, y=yearly[element])],
        layout_title_text="Hide Y-axis Tick Labels",
        layout = {
            'xaxis': {'title': 'x-axis', 'visible': False, 'showticklabels': True},
            
            'yaxis': {'title': 'y-axis', 'visible': True, 'showticklabels': False},

            # specify margins in px
            'margin': dict(
                l = 20, # left
                r = 15, # right
                t = 30, # top
                b = 40, # bottom
            ),
        },
    )
    return fig

app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [27/Nov/2022 08:43:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2022 08:43:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2022 08:43:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2022 08:43:18] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Nov/2022 08:43:18] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Nov/2022 08:43:18] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Nov/2022 08:43:18] "GET /_favicon.ico?v=2.6.2 HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2022 08:43:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2022 08:43:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2022 08:43:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2022 08:43:22] "POST /_dash-update-component HTTP/1.1" 200